In [1]:
# prompt: connect to my drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
# prompt: read in /content/drive/My Drive/data/df_notes_demographics.csv

import pandas as pd

train_notes_df= pd.read_csv("df_notes.csv")


In [ ]:
train_notes_df.head(2)

In [2]:
url = 'https://raw.githubusercontent.com/hibaahsan/MIMIC-SBDH/main/MIMIC-SBDH.csv'
train_ann_df= pd.read_csv(url)

In [3]:
alcohol_level_dict = {
    0: 0,
    1: 1,
    2: 1,
    3: 1,
    4: 1
}

environment_level_dict = {
    0: 0,
    1: 1,
    2: 1
}

train_set_df = train_notes_df.merge(train_ann_df, left_on="ROW_ID", right_on="row_id")
train_set_df

# recode the levels of 'behavior_alcohol' based on the alcohol level dict
train_set_df.loc[:, 'alcohol_binary'] = train_set_df.loc[:, 'behavior_alcohol'].map(alcohol_level_dict)

# recode the levels of 'sdoh_environment' based on the environment level dict
train_set_df.loc[:, 'environment_binary'] = train_set_df.loc[:, 'sdoh_environment'].map(environment_level_dict)

# if we have documentation of community present OR absent, code the binary version as 1 else 0
train_set_df.loc[:, 'community_binary'] = (train_set_df.sdoh_community_present == 1) | (train_set_df.sdoh_community_absent == 1)
train_set_df.loc[:, 'community_binary'] = train_set_df.loc[:, 'community_binary'].astype(int)

In [4]:
train_set_df.behavior_alcohol.value_counts()

3    2444
1    2077
0    1657
2     515
4     332
Name: behavior_alcohol, dtype: int64

In [5]:
train_set_df[['sdoh_community_present', 'sdoh_community_absent']].value_counts()

sdoh_community_present  sdoh_community_absent
1                       0                        3878
0                       0                        2363
1                       1                         585
0                       1                         199
dtype: int64

In [ ]:
train_set_df.environment_binary.value_counts()

environment_binary
1    4420
0    2605
Name: count, dtype: int64

In [ ]:
train_set_df.community_binary.value_counts()

community_binary
1    4662
0    2363
Name: count, dtype: int64

In [ ]:
train_set_df.alcohol_binary.value_counts()

alcohol_binary
1    5368
0    1657
Name: count, dtype: int64

In [ ]:
train_set_df.head()

,Unnamed: 0,ROW_ID,TEXT,row_id,sdoh_community_present,sdoh_community_absent,sdoh_education,sdoh_economics,sdoh_environment,behavior_alcohol,behavior_tobacco,behavior_drug,alcohol_binary,environment_binary,community_binary
0,0,37988,Admission Date: [**2166-6-5**] D...,37988,0,0,0,0,1,2,3,0,1,1,0
1,1,37282,Admission Date: [**2109-12-23**] ...,37282,1,1,0,1,0,2,0,0,1,0,1
2,2,26313,Admission Date: [**2114-12-17**] Discha...,26313,0,0,0,0,0,3,2,0,1,0,0
3,3,13852,Admission Date: [**2112-5-16**] ...,13852,0,0,0,0,0,1,0,0,1,0,0
4,4,51031,Admission Date: [**2155-2-26**] ...,51031,1,1,0,0,1,0,0,0,0,1,1


1. Data Preprocessing

In [6]:
import pandas as pd
import re
def extract_section(text, start_keyword):
    """
    Extracts the section from the text starting with a keyword until a blank line or
    another major section header is detected.
    """
    # Convert to lowercase for case-insensitive matching, but work with original for extraction
    text_lower = text.lower()
    start_keyword = start_keyword.lower()

    try:
        start_index = text_lower.index(start_keyword) + len(start_keyword)
        # Find the end index by looking for two newline characters indicating a new section or end of the note
        end_index = text_lower.find('\n\n', start_index)
        if end_index == -1:  # If no double newline is found, take the rest of the text
            end_index = len(text)
        return text[start_index:end_index].strip()
    except ValueError:
        # Return None if the section isn't found
        return None

def clean_text(text):
    """
    Performs basic cleaning of text such as removing extra spaces, special characters, etc.
    """
    if text is not None:
        text = re.sub(r'\[\*\*.*?\*\*\]', '', text)  # Remove de-identified placeholders
        text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
    return text




In [7]:
# Define keywords to mark the start and end of the section of interest
# Define keywords to mark the start and end of the section of interest, maintain original case for matching
start_keyword = "Social History:"

# Extract and clean the Social History section from each row
train_set_df['processed_text'] = train_set_df['TEXT'].apply(
    lambda x: clean_text(extract_section(x, start_keyword))
)
# Count how many rows have None or missing 'processed_text'
missing_count = train_set_df['processed_text'].isnull().sum()

print(f"Number of records with missing 'Social History' sections: {missing_count}")


Number of records with missing 'Social History' sections: 0


In [8]:
# Calculate the number of words in each row of 'processed_text'
train_set_df['word_count'] = train_set_df['processed_text'].apply(lambda x: len(x.split()))

# Get descriptive statistics of the word count distribution
train_set_df['word_count'].describe()


count    7025.000000
mean       25.492242
std        19.714779
min         0.000000
25%        12.000000
50%        21.000000
75%        34.000000
max       234.000000
Name: word_count, dtype: float64

Model Selection and Training

In [9]:
pip install transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 38.3 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import DistilBertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        # Encoding the text for DistilBERT
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Assume train_set_df is your DataFrame and already has 'processed_text'
texts = train_set_df['processed_text'].tolist()
labels = train_set_df[['alcohol_binary', 'environment_binary', 'community_binary']].values

# Create the dataset
dataset = TextDataset(texts, labels, tokenizer)

# DataLoader for batching
loader = DataLoader(dataset, batch_size=16, shuffle=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [11]:
import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np
# Modify the config to have 3 labels
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
from sklearn.model_selection import train_test_split

# Assuming 'texts' and 'labels' are already defined as shown previously
# Split data into training and temporary dataset first (e.g., 80% train, 20% temp)
texts_train, texts_temp, labels_train, labels_temp = train_test_split(
    texts, labels, test_size=0.2, random_state=42)

# Split the temporary dataset into validation and test sets (e.g., 50% each of temp)
texts_val, texts_test, labels_val, labels_test = train_test_split(
    texts_temp, labels_temp, test_size=0.5, random_state=42)

# Print out the distribution of the datasets
print(f"Training set size: {len(texts_train)}")
print(f"Validation set size: {len(texts_val)}")
print(f"Test set size: {len(texts_test)}")


Training set size: 5620
Validation set size: 702
Test set size: 703


In [13]:
# Create datasets for training, validation, and testing
train_dataset = TextDataset(texts_train, labels_train, tokenizer)
val_dataset = TextDataset(texts_val, labels_val, tokenizer)
test_dataset = TextDataset(texts_test, labels_test, tokenizer)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [14]:
subset_size = int(0.1 * len(train_dataset))
subset_train_dataset = Subset(train_dataset, indices=range(subset_size))
subset_train_loader = DataLoader(subset_train_dataset, batch_size=16, shuffle=True)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.BCEWithLogitsLoss()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
def train_and_evaluate(model, train_loader, val_loader, optimizer, device, criterion, epochs=1):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for d in train_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Average Training Loss: {total_loss / len(train_loader):.2f}")
        # Evaluate and print performance
        train_acc, train_auc = evaluate(model, train_loader, device)
        val_acc, val_auc = evaluate(model, val_loader, device)
        print(f"Train Accuracy: {train_acc}, Train AUC: {train_auc}")
        print(f"Validation Accuracy: {val_acc}, Validation AUC: {val_auc}")

def evaluate(model, data_loader, device):
    model.eval()
    final_targets = []
    final_outputs = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            final_outputs.extend(outputs.detach().cpu().numpy())
            final_targets.extend(targets.detach().cpu().numpy())
    final_outputs = np.array(final_outputs)
    final_targets = np.array(final_targets)
    accuracies = [accuracy_score(final_targets[:, i], final_outputs[:, i] > 0) for i in range(final_targets.shape[1])]
    aucs = [roc_auc_score(final_targets[:, i], final_outputs[:, i]) for i in range(final_targets.shape[1])]
    return accuracies, aucs
train_and_evaluate(model, subset_train_loader, val_loader, optimizer, device, criterion, epochs=3)


Epoch 1, Average Training Loss: 0.53
Train Accuracy: [0.7597864768683275, 0.8790035587188612, 0.8807829181494662], Train AUC: [0.8269581056466304, 0.9574021278075239, 0.9554233278371209]
Validation Accuracy: [0.7521367521367521, 0.8561253561253561, 0.8689458689458689], Validation AUC: [0.7187826541274818, 0.9433638985750571, 0.9517856985659832]
Epoch 2, Average Training Loss: 0.32
Train Accuracy: [0.7935943060498221, 0.9697508896797153, 0.9572953736654805], Train AUC: [0.9461184838234019, 0.9964397468882554, 0.9955839128252921]
Validation Accuracy: [0.7678062678062678, 0.9387464387464387, 0.9259259259259259], Validation AUC: [0.8539925113200975, 0.9821344009643119, 0.9799642596195322]
Epoch 3, Average Training Loss: 0.19
Train Accuracy: [0.9359430604982206, 0.9822064056939501, 0.9875444839857651], Train AUC: [0.9760083268279989, 0.999554968361032, 0.9994264821851029]
Validation Accuracy: [0.8803418803418803, 0.9358974358974359, 0.9601139601139601], Validation AUC: [0.9233607628004179, 

In [16]:
torch.save(model.state_dict(), 'model_state_dict.pt')

# Optionally, save the entire model
torch.save(model, 'full_model.pt')

In [1]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = torch.load('full_model.pt')
model = model.to(device)
model.eval()

# If you saved just the state dictionary (assuming the model architecture is known)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
model.load_state_dict(torch.load('model_state_dict.pt'))
model = model.to(device)
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [2]:
# Prepare your new DataFrame
new_df =  pd.read_csv("df_test_filtered.csv")
new_df = new_df[new_df['processed_text'].notnull()]
# Load your new data

# Prepare the dataset
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create dataset and dataloader
inference_dataset = InferenceDataset(new_df['processed_text'].tolist(), tokenizer)
inference_loader = DataLoader(inference_dataset, batch_size=16, shuffle=False)


In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
# import pandas as pd
# from transformers import DistilBertTokenizer

# # Assuming the tokenizer and model are already loaded and set up
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = torch.load('full_model.pt')
# model.eval()
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# class InferenceDataset(Dataset):
#     def __init__(self, texts, tokenizer, max_length=512):
#         self.tokenizer = tokenizer
#         self.texts = texts
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         encoding = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_length,
#             return_token_type_ids=False,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors='pt',
#         )
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#         }

# # Create dataset and loader
# inference_dataset = InferenceDataset(new_texts, tokenizer)
# inference_loader = DataLoader(inference_dataset, batch_size=16, shuffle=False)


In [ ]:
def get_predictions(model, data_loader):
    model.eval()
    final_outputs = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            outputs = torch.sigmoid(outputs)
            outputs = (outputs > 0.5).long()  # Converting probabilities to 0 or 1
            final_outputs.extend(outputs.cpu().numpy())
    return final_outputs


predictions = get_predictions(model, inference_loader)


new_df[['alcohol_label', 'environment_label', 'community_label']] = pd.DataFrame(predictions, index=new_df.index)

len(new_df)


In [ ]:
new_df.to_csv('predictions_with_labels.csv', index=False)

In [ ]:
# we do not use the code under for the last version of inference 

In [ ]:
# check the inference 


def get_predictions(model, data_loader, threshold=0.5):
    model = model.to(device)
    final_outputs = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).logits
            outputs = torch.sigmoid(outputs)  # Convert logits to probabilities
            outputs = (outputs > threshold).long()  # Convert probabilities to binary labels
            final_outputs.extend(outputs.detach().cpu().numpy())
    return final_outputs

# Get predictions with binary classification
predictions = get_predictions(model, inference_loader)

# Append predictions to DataFrame
new_df[['alcohol_label', 'environment_label', 'community_label']] = pd.DataFrame(predictions, index=new_df.index)


In [ ]:
output_df = new_df[['ROW_ID', 'alcohol_label', 'environment_label', 'community_label']]


output_df.to_csv('predictions.csv', index=False)


In [ ]:
def save_checkpoint(model, optimizer, filename='checkpoint.pth'):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, filename)

# After training on the initial subset
save_checkpoint(model, optimizer, filename='checkpoint_10pct.pth')

In [ ]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import AdamW
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = BCEWithLogitsLoss()

def train_epoch(model, data_loader, optimizer, device, criterion):
    model = model.train()

    losses = []
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=None
        )

        logits = outputs.logits
        loss = criterion(logits, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return np.mean(losses)
def evaluate(model, data_loader, device):
    model = model.eval()
    final_targets = []
    final_outputs = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            ).logits

            final_outputs.extend(outputs.detach().cpu().numpy())
            final_targets.extend(targets.detach().cpu().numpy())

    final_outputs = np.array(final_outputs)
    final_targets = np.array(final_targets)

    accuracies = [accuracy_score(final_targets[:, i], final_outputs[:, i] > 0) for i in range(final_targets.shape[1])]
    aucs = [roc_auc_score(final_targets[:, i], final_outputs[:, i]) for i in range(final_targets.shape[1])]

    return accuracies, aucs
train_accuracies = []
train_aucs = []
val_accuracies = []
val_aucs = []

for epoch in range(3):
    train_loss = train_epoch(model, train_loader, optimizer, device, criterion)
    train_acc, train_auc = evaluate(model, train_loader, device)
    val_acc, val_auc = evaluate(model, val_loader, device)

    train_accuracies.append(train_acc)
    train_aucs.append(train_auc)
    val_accuracies.append(val_acc)
    val_aucs.append(val_auc)

    print(f"Epoch {epoch + 1}, Training Loss: {train_loss:.2f}")
    print(f"Train Acc: {train_acc}, Train AUC: {train_auc}")
    print(f"Val Acc: {val_acc}, Val AUC: {val_auc}")

KeyboardInterrupt: 

In [ ]:
epochs = range(1, 4)
fig, ax = plt.subplots(2, 1, figsize=(10, 10))

# Plot accuracies
for i in range(3):
    ax[0].plot(epochs, [acc[i] for acc in train_accuracies], label=f'Train Class {i+1} Accuracy')
    ax[0].plot(epochs, [acc[i] for acc in val_accuracies], label=f'Val Class {i+1} Accuracy', linestyle='--')
ax[0].set_title('Accuracy over Epochs by Class')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend()

# Plot AUCs
for i in range(3):
    ax[1].plot(epochs, [auc[i] for auc in train_aucs], label=f'Train Class {i+1} AUC')
    ax[1].plot(epochs, [auc[i] for auc in val_aucs], label=f'Val Class {i+1} AUC', linestyle='--')
ax[1].set_title('AUC over Epochs by Class')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('AUC')
ax[1].legend()

plt.tight_layout()
plt.show()
